### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!


- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv

import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1711bb920>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1785399a0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
!pip install langchain_core

In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

message = [
    SystemMessage(content="Translate the following from English to Telugu"),
    HumanMessage(content="Hello how are you?"),

]

results = model.invoke(message)

In [9]:
results

AIMessage(content='Hello how are you? translates to **"హలో, ఎలా ఉన్నారా?" (Halo, ela unnarā?)** in Telugu. \n\nHere\'s a breakdown:\n\n* **హలో (Halo)**: Hello\n* **ఎలా (Ela)**: How\n* **ఉన్నారా (Unnarā?)**: Are you (formal, respectful) \n\n\n\nLet me know if you have any other phrases you\'d like translated!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 98, 'prompt_tokens': 21, 'total_tokens': 119, 'completion_time': 0.178181818, 'prompt_time': 0.001346659, 'queue_time': 0.079443267, 'total_time': 0.179528477}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--76a39b98-dc90-4ce7-8e02-f31af1557f0c-0', usage_metadata={'input_tokens': 21, 'output_tokens': 98, 'total_tokens': 119})

In [10]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(results)

'Hello how are you? translates to **"హలో, ఎలా ఉన్నారా?" (Halo, ela unnarā?)** in Telugu. \n\nHere\'s a breakdown:\n\n* **హలో (Halo)**: Hello\n* **ఎలా (Ela)**: How\n* **ఉన్నారా (Unnarā?)**: Are you (formal, respectful) \n\n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [12]:
### Using LCEL- chain the components

chain = model | parser
chain.invoke(message)

'Here are a few ways to say "Hello, how are you?" in Telugu:\n\n* **నమస్తే, మీరు ఎలా ఉన్నారు?** (Namaste, meeru ela unnaru?) - This is a formal and respectful way to say hello and ask how someone is doing.\n* **హేలో, ఎలా ఉన్నారు?** (Hello, ela unnaru?) - This is a more casual way to say hello and ask how someone is doing.\n* **ఏమైంది?** (Eemayindi?) - This is a very informal way to say "What\'s up?" or "How are you?".\n\n\nIt depends on the context and your relationship with the person you\'re speaking to. \n\n'

In [15]:
### Prompt Templates

from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [17]:
results = prompt.invoke({"language":"Telugu","text":"Hello"})


In [18]:
results.to_messages()

[SystemMessage(content='Translate the following into Telugu:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [23]:
##Chaining together components with LCEL

chain = prompt | model | parser
chain.invoke({"language":"Telugu","text":"Hello"})

'Hello in Telugu is: \n\n**హలో** (Hālo) \n\n\nYou can also say:\n\n* **నమస్కారం** (Namskaaram) -  This is a more formal greeting.\n* **ప్రైవేట్** (Praiveet) -  This is a more casual greeting.\n'